# Imports

In [1]:
#%pip install pandas
#%pip install numpy
#%pip install sklearn
#%pip install xgboost
#%pip install lightgbm

#For additional data pre-proccessing & augmentation
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
import random

#For splitting and formating data for training
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#For evaluating models
from sklearn.metrics import accuracy_score, classification_report

#Models
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 1. Dataset Preparation

## Load & Augment Pre-processed Dataset

In [5]:
df_o = pd.read_pickle('cleaned_cases.pkl')

In [6]:
def synonym_replacement(text):
    if not text:
        return ''
    
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replacements = max(1, int(0.1 * len(words)))  # Replace around 10% of the words
    
    replacements = 0
    for random_word in random_word_list:
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words = [synonym if word == random_word else word for word in new_words]
            replacements += 1
        if replacements >= num_replacements:
            break

    return ' '.join(new_words)

def mirror_case(row):
    return {
        'name': f"{row['name']} (Mirrored)",
        'first_party': row['second_party'],
        'second_party': row['first_party'],
        'winning_party': row['winning_party'],
        'Facts': synonym_replacement(row['Facts']),
        'question': synonym_replacement(row['question']),
        'conclusion': synonym_replacement(row['conclusion']),
        'winner_index': 1 - row['winner_index']
    }

mirrored = []
for index, row in df_o.iterrows():
    mirrored.append(mirror_case(row))
df_m = pd.DataFrame(mirrored)

df = pd.concat([df_o, df_m])

In [7]:
print(f'Cases: {len(df)}')
pd.set_option('display.max_colwidth', None)
df.sample()

Cases: 6928


,name,first_party,second_party,winning_party,question,conclusion,winner_index,Facts
1749,Brandenburg v. Ohio (Mirrored),State of Ohio,Clarence Brandenburg,Brandenburg,"Did Ohio's criminal syndicalism law, prohibiting public speech that advocates assorted illegal activities, violate Brandenburg's right to free speech as protected by the First and Fourteenth Amendments?","The Court's Per Curiam opinion held that the Ohio law violated Brandenburg's right to free speech. The Court used angstrom two-pronged test to measure speech acts: (1) speech can be prohibited if it is ""directed at motivate or producing imminent lawless action"" and (2) it is ""likely to incite or produce such action."" The criminal syndicalism act made illegal the advocacy and teaching of doctrines while ignoring whether or not that advocacy and teaching would actually incite imminent lawless action. The failure to brand this differentiation rendered the law overly broad and in misdemeanor of the Constitution.",1,"Brandenburg, a leader in the Ku Klux Klan, made a speech at a Klan rally and was later convicted under an Ohio criminal syndicalism law. The law made illegal recommend ""crime, sabotage, violence, or improper methods of terrorism arsenic a means of accomplishing industrial or political reform,"" arsenic well arsenic assembling ""with any society, group, or assemblage of persons formed to Teach or advocate the doctrines of criminal syndicalism."""


# 2. AI Judge

Splitting and Preparing Data for Neural Network Training

In [29]:
X_train_party1_text, X_test_party1_text, \
X_train_party2_text, X_test_party2_text, \
X_train_facts_text, X_test_facts_text, \
y_train, y_test = train_test_split(
    df['first_party'],
    df['second_party'],
    df['Facts'],
    df['winner_index'],
    test_size=0.2
)

In [30]:
vectorizer_party1 = TfidfVectorizer()
vectorizer_party2 = TfidfVectorizer()
vectorizer_facts = TfidfVectorizer()

X_train_party1 = vectorizer_party1.fit_transform(X_train_party1_text)
X_test_party1 = vectorizer_party1.transform(X_test_party1_text)

X_train_party2 = vectorizer_party2.fit_transform(X_train_party2_text)
X_test_party2 = vectorizer_party2.transform(X_test_party2_text)

X_train_facts = vectorizer_facts.fit_transform(X_train_facts_text)
X_test_facts = vectorizer_facts.transform(X_test_facts_text)

# Combine features
X_train = np.hstack([X_train_party1.toarray(), X_train_party2.toarray(), X_train_facts.toarray()])
X_test = np.hstack([X_test_party1.toarray(), X_test_party2.toarray(), X_test_facts.toarray()])


### Evaluating KNN

In [39]:
knn_classifier = KNeighborsClassifier(n_neighbors= 5, weights = 'distance')
knn_classifier.fit(X_train, y_train)

knn_predictions = knn_classifier.predict(X_test)
print("KNN Classifier Accuracy: ", accuracy_score(y_test, knn_predictions))

KNN Classifier Accuracy:  0.6464646464646465


### Evaluating Naive Bayes

In [43]:
nb_classifier = MultinomialNB(alpha = 5)
nb_classifier.fit(X_train, y_train)

nb_predictions = nb_classifier.predict(X_test)
print("Naive Bayes Classifier Accuracy: ", accuracy_score(y_test, nb_predictions))


Naive Bayes Classifier Accuracy:  0.6392496392496393


### Evaluating SVM

In [44]:
svm_classifier = LinearSVC(C=0.7, intercept_scaling=0.1, loss='squared_hinge', dual='auto')
svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)
print("SVM Classifier Accuracy: ", accuracy_score(y_test, svm_predictions))

SVM Classifier Accuracy:  0.6955266955266955


### Evaluating XG Boost

In [45]:
xgb_classifier = XGBClassifier(random_state=7)
xgb_classifier.fit(X_train, y_train)

y_pred_xgb = xgb_classifier.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Classifier Test Accuracy: {xgb_accuracy}")


XGBoost Classifier Test Accuracy: 0.7373737373737373


### LightGBM

In [46]:
lgbm_classifier = LGBMClassifier(random_state=7, verbose = -1)
lgbm_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred_lgbm = lgbm_classifier.predict(X_test)
lgbm_accuracy = accuracy_score(y_test, y_pred_lgbm)
print(f"LightGBM Classifier Test Accuracy: {lgbm_accuracy}")


LightGBM Classifier Test Accuracy: 0.7251082251082251


### Log Regression

In [47]:
log_classifier = LogisticRegression(max_iter=10000, random_state=7)
log_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred_log_reg = log_classifier.predict(X_test)
log_reg_accuracy = accuracy_score(y_test, y_pred_log_reg)
print(f"Logistic Regression Test Accuracy: {log_reg_accuracy}")

Logistic Regression Test Accuracy: 0.7056277056277056


### Voting Classifier Test

In [48]:
voting_classifier = VotingClassifier(
    estimators=[
        ('knn', knn_classifier),
        ('nb', nb_classifier),
        ('svm', svm_classifier),
        ('log', log_classifier),
        ('lgbm', lgbm_classifier),
        ('xgb', xgb_classifier)
    ],
    voting='hard'
)

# Train the voting classifier
voting_classifier.fit(X_train, y_train)
y_test_pred = voting_classifier.predict(X_test)

print('voting_classifier - Test\n', classification_report(y_test, y_test_pred, zero_division=0))

voting_classifier - Test
               precision    recall  f1-score   support

           0       0.68      0.76      0.72       670
           1       0.75      0.67      0.71       716

    accuracy                           0.72      1386
   macro avg       0.72      0.72      0.72      1386
weighted avg       0.72      0.72      0.72      1386



# 3. Demo

In [50]:
def predict(party1, party2, facts):
    X_party1 = vectorizer_party1.transform([party1]).todense()
    X_party2 = vectorizer_party2.transform([party2]).todense()
    X_facts = vectorizer_facts.transform([facts]).todense()

    # Combine the features
    X = np.asarray(np.hstack([X_party1, X_party2, X_facts]))

    # Predict the outcome using the voting classifier
    win_index = voting_classifier.predict(X)[0]
    if (win_index == 0): 
        return party1
    else:
         return party2

In [55]:
out = predict(
    'American Hospital Association, et al.',
    'Xavier Becerra, Secretary of Health and Human Services, et al.',  
    'The federal government reimburses hospitals for providing outpatient care to patients insured by Medicare Part B. Until recently, the government reimbursed all hospitals at a uniform rate for providing covered drugs. In 2018, the Department of Health and Human Services (HHS) reduced the reimbursement rates for certain types of hospitals (known as “340B hospitals”) because those hospitals can obtain the covered drugs far more cheaply than other hospitals can. HHS reasoned that it should not reimburse hospitals more than they paid to acquire the drugs. Several 340B hospitals and hospital associations affected by the decision filed a lawsuit challenging HHS’s decision to lower reimbursement rates. The district court ruled that HHS had exceeded its statutory authority by reducing drug reimbursement rates for 340B hospitals, but the U.S. Court of Appeals for the D.C. Circuit reversed, finding that HHS’s decision is based on a reasonable interpretation of the statute.')
print(f'Expecting American Hospital Association, et al\n AI Judge rules in favor of {out}')

Expecting American Hospital Association, et al
 AI Judge rules in favor of American Hospital Association, et al.
